# Manipulando resultados da busca com Pandas

Este arquivo exemplifica o uso da biblioteca de análise de dados [Pandas](https://pandas.pydata.org/) para o estudo de metadados obtidos com a ferramenta metricapes.

## Carregamento

In [ ]:
import pandas

df = pandas.read_csv('susten.tsv', delimiter='\t', encoding='utf-8')

print(f"Total de trabalhos carregados: {len(df)}.")
df.head()

## Conversão em formato BibTeX

O código abaixo gera um arquivo de bibliografia no formato [BibTeX](https://www.bibtex.org/). Embora o formato preveja a inclusão de teses e dissertações, aqui as entradas são formatadas como artigos (`@article`) para compatibilidade com a ferramenta de análise bibliométrica [Bibliometrix](https://bibliometrix.org/).

In [ ]:
teses = df.to_dict('records')

def bib_article(tese):
    aut_fmt = tese['autor'].rsplit(' ', 1)[1] + ", " + tese['autor'].rsplit(' ', 1)[0]
    aut_fmt += ' and ' + tese['orientacao'].rsplit(' ', 1)[1] + ', ' + tese['orientacao'].rsplit(' ', 1)[0] if type(tese['orientacao']) is str else ''
    bib = (f"@article{{ CAPES:{tese['id']},\n"
           f"Author = {{{aut_fmt}}},\n"
           f"Title = {{{tese['titulo']}}},\n"
           "Journal = {},\n"
           f"Year = {{{tese['dataDefesa'][:4]}}},\n"
           f"Publisher = {{{tese['instituicao']}}},\n"
           "Type = {Article},\n"
           "Language = {Portuguese},\n"
           f"Keywords = {{{tese['palavrasChave'].replace(';', '; ') if type(tese['palavrasChave']) is str else ''}}},\n"
           f"Research-Areas = {{{tese['areaConcentracao']}}},\n"
           f"Affiliation = {{{tese['instituicao']}}},\n"
           f"Unique-ID = {{CAPES:{tese['id']}}},\n"
           "}\n\n")
    return bib

bibrecords = [bib_article(tese) for tese in teses]

with open('todos.bib', 'w', encoding='utf-8') as f:
    for r in bibrecords:
        f.write(r)

# Principais Programas

Os 10 (nomes de) programas de pós-graduação mais frequentes no conjunto de dados: 

In [ ]:

df[['nomePrograma', 'id']].groupby(by='nomePrograma').count().nlargest(n=10, columns=['id'])

## Verificação de consistência

Alguns campos como `palavrasChave`, `resumo` e `orientacao` são obtidos por meio do link existente na base da CAPES e presente no campo `link`. Abaixo verificamos se há trabalhos que possuem links mas nos quais não foram carregadas as palavras chave, e exibimos os links para verificação.

In [ ]:
df.link.loc[~df['link'].str.match('undefined') & df['palavrasChave'].isna()].tolist()

## Salvar alterações

Caso seja desejável salvar modificações nos arquivos TSV, o código abaixo exemplifica como fazê-lo de modo compatível com o formato utilizado.

In [ ]:
df.to_csv('susten_copia.tsv', sep='\t', encoding='utf-8', index=False)